- `N` represents the **total number of items** in the search space (unsorted database).
- In Grover's Algorithm, we are searching for a **marked element** among these `N` elements.


In [2]:
N = 8

In [3]:
from math import ceil, log2
n = ceil(log2(N))

In [4]:
n

3

- We need **`n` qubits** to represent `N` possible states.
   - Since a quantum register of `n` qubits can represent $2^n$ states, we need to solve:
     
     $
     2^n \geq N
     $  
   - Taking the **log base 2** gives:
     
     $
     n = \lceil \log_2(N) \rceil
     $  
   - For `N = 8`, we get:
     
     $
     n = \lceil \log_2(8) \rceil = \lceil 3 \rceil = 3
     $  
   - **Meaning:** We need **3 qubits** to represent `N = 8` elements.

In [5]:
from random import randint
winner_index = randint(0,N-1)

In [6]:
winner_index

3

In [7]:
winner_state = bin(winner_index)[2:].zfill(n)

In [8]:
winner_state

'011'

- `bin(winner_index)[2:]` converts the decimal number into a **binary string** and removes the `"0b"` prefix.
- `.zfill(n)` ensures that the binary string has exactly `n` bits (padding with leading zeros if needed).
- If `winner_index = 3`, the correct solution is at index `3`.
- This variable is expected to store the **index of the correct solution** (the item we are searching for).
- 
**Example: If `winner_index = 3` (decimal 3 in 3-bit binary)**  
   ```python
   bin(3)  # '0b11'
   bin(3)[2:]  # '11'
   '11'.zfill(3)  # '011' (adds one leading zero)


**Why is this useful in Grover’s Algorithm?**
- The algorithm searches for a special state (the **marked state**).
- `winner_state` is the **target quantum state** that Grovers Algorithm will amplify.